In [2]:
# -*- coding: utf-8 -*-
# load datasets
from datasets import load_dataset
import pandas as pd
import warnings
from utils import filter_duplicates_disagreement, create_post2summaries_ratings, create_dict_comparision, process_comparisons_df, update_summary_t2id
import worker_modeling

warnings.filterwarnings('ignore')

comparisons = load_dataset("openai/summarize_from_feedback", name="comparisons")
axis = load_dataset("openai/summarize_from_feedback", name="axis")

comparisons_train = comparisons["train"]
comparisons_val = comparisons["validation"]
axis_val = axis["validation"]


comparisons_validation_dict = create_dict_comparision(comparisons_val)
comparisons_train_validation_dict = create_dict_comparision(comparisons_train)
comparisons_train_validation_dict.update(comparisons_validation_dict)

rating_validation_overlap, repeated_summaries, summary_text2id = create_post2summaries_ratings(axis_val)

complete_summary_t2id = update_summary_t2id(comparisons_train, summary_text2id)
comparisons_train_df = process_comparisons_df(comparisons_train,complete_summary_t2id)
comp_train_df_dup = filter_duplicates_disagreement(comparisons_train_df)

#full validation set
complete_summary_t2id = update_summary_t2id(comparisons_val, summary_text2id)
comparisons_validation_df = process_comparisons_df(comparisons_val,complete_summary_t2id)
comp_val_df_dup = filter_duplicates_disagreement(comparisons_validation_df)

In [3]:
total_annotated_pairs = len(comparisons_train)+len(comparisons_val)
total_annotated_pairs

178944

In [4]:
dis_and_dup= total_annotated_pairs -(len(comp_val_df_dup)+len(comp_train_df_dup)) 
dis_and_dup

41287

In [5]:
len_training = len(comp_train_df_dup)
validation_set = comp_val_df_dup[:-10000]
test_set = comp_val_df_dup[-10000:]
len_valid= len(validation_set)
len_testing = len(test_set)
print(len_training,len_valid ,len_testing)

70042 57615 10000


In [10]:
choice_distribution_train = comp_train_df_dup['choice'].value_counts()
choice_distribution_val = comp_val_df_dup['choice'].value_counts()

combined_choice_distribution = choice_distribution_train.add(choice_distribution_val, fill_value=0)

print(combined_choice_distribution)


choice
1    69447
0    68210
Name: count, dtype: int64


In [13]:
combined_df = pd.concat([comp_train_df_dup, comp_val_df_dup])

unique_workers_count = combined_df['worker'].nunique()

print(f"Total unique workers: {unique_workers_count}")


Total unique workers: 73
